<a href="https://colab.research.google.com/github/jonvthvn90/Project/blob/main/PathCNN_Wookjin_Choi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PathCNN

Wookjin Choi

## Download source code

In [ ]:
!git clone https://github.com/wchoi-student/PathCNN.git

Cloning into 'PathCNN'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 102 (delta 41), reused 5 (delta 0), pack-reused 0
Receiving objects: 100% (102/102), 7.88 MiB | 20.33 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [ ]:
!ls PathCNN
%cd PathCNN

data  PathCNN_GradCAM_modeling.py  PathCNN.py		     README.md
img   PathCNN_GradCAM.py	   Perceptron_and_DNN.ipynb  requirements.txt
/content/PathCNN


## Install required packages

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 86.3 MB 18 kB/s 
     |████████████████████████████████| 377 kB 40.3 MB/s 
     |████████████████████████████████| 2.9 MB 40.7 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 3.8 MB 40.3 MB/s 
     |████████████████████████████████| 449 kB 27.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=0f79f9c58cd95993f304273136108cdf74e8b94adc7ca3d7d9134edb488f4134
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: ten

In [ ]:
!python PathCNN_GradCAM_modeling.py

Using TensorFlow backend.
x_train shape: (287, 146, 6, 1)
287 train samples
287 validation samples
287 test samples
2021-11-01 22:44:00.663727: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-01 22:44:00.667360: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-11-01 22:44:00.667625: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a4465fb2c0 executing computations on platform Host. Devices:
2021-11-01 22:44:00.667661: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 146, 6, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (

In [ ]:
!python PathCNN_GradCAM.py

Using TensorFlow backend.
(343, 730)
test
[1.         0.23706897]
2021-11-01 22:44:52.753050: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-01 22:44:52.757268: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-11-01 22:44:52.757511: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5602241c2840 executing computations on platform Host. Devices:
2021-11-01 22:44:52.757550: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 146, 6, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 6, 32)        320       
______________

## Noisy data test

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Input
from keras import backend as K
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
#from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from keras import regularizers
from keras.layers.core import *
from keras.models import Model
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix

pca_exp = pd.read_excel("data/PCA_EXP.xlsx", header=None)
pca_cnv = pd.read_excel("data/PCA_CNV.xlsx", header=None)
pca_mt = pd.read_excel("data/PCA_MT.xlsx", header=None)
clinical = pd.read_excel("data/Clinical.xlsx")

n = len(pca_exp)  # sample size: number of Pts
path_n = 146  # number of pathways
pc = 5  # number of PCs

# data creation-EXP
pca_exp = pca_exp.to_numpy()
exp_data = np.zeros((n, path_n, pc))
for i in range(n):
    for j in range(path_n):
        exp_data[i, j, :] = pca_exp[i, j * pc:(j + 1) * pc]

# data creation-CNV
pca_cnv = pca_cnv.to_numpy()
cnv_data = np.zeros((n, path_n, pc))
for i in range(n):
    for j in range(path_n):
        cnv_data[i, j, :] = pca_cnv[i, j * pc:(j + 1) * pc]

# data creation-MT
pca_mt = pca_mt.to_numpy()
mt_data = np.zeros((n, path_n, pc))
for i in range(n):
    for j in range(path_n):
        mt_data[i, j, :] = pca_mt[i, j * pc:(j + 1) * pc]

# data merge: mRNA expression, CNV, and MT with a specific number of PCs
no_pc = 2  # use the first 2PCs among 5 PCs
all_data = np.zeros((n, path_n, no_pc * 3))
for i in range(n):
        all_data[i, :, :] = np.concatenate((exp_data[i, :, 0:no_pc], cnv_data[i, :, 0:no_pc], mt_data[i, :, 0:no_pc]),axis=1)

clinical = clinical.to_numpy()
age = clinical[:, 4]
survival = clinical[:, 5]
os_months = clinical[:, 6]
idx0 = np.where((survival == 1) & (os_months <= 24))
idx1 = np.where(os_months > 24)

bio_data=clinical[:, 7:10]
sigma = 2
mu = 0.01
all_data = all_data[:, :, :] + np.random.randn(*all_data.shape)*sigma + mu # adding gaussian noise

data_0 = all_data[idx0, :, :]
data_0 = data_0[0, :, :, :]
data_1 = all_data[idx1, :, :]
data_1 = data_1[0, :, :, :]

age_0 = age[idx0]
age_1 = age[idx1]

bio_data_0 = bio_data[idx0, : ]
bio_data_0 = bio_data_0[0, :, :]
bio_data_1 = bio_data[idx1,:]
bio_data_1 = bio_data_1[0, :, :]

outcomes_0 = np.zeros(len(idx0[0]))
outcomes_1 = np.ones(len(idx1[0]))

## data merge
age_r = np.concatenate((age_0, age_1))
data = np.concatenate((data_0, data_1))
outcomes = np.concatenate((outcomes_0, outcomes_1))
bio_data_r=np.concatenate((bio_data_0, bio_data_1))

## DEEP LEARNING
batch_size = 64
epochs = 30

num_classes = 2
num_runs = 30
k_fold = 5

# input image dimension
img_rows, img_cols = 146, 6

# use all data for modeling
train1 = train = range(data.shape[0])
test = range(data.shape[0])
validation = range(data.shape[0])

x_train = data[train, :, :]
y_train = outcomes[train]

x_validation = data[validation, :, :]
y_validation = outcomes[validation]

x_test = data[test, :, :]
y_test = outcomes[test]

# age
age_r1=age_r[train1]
age_train = age_r1[train]
age_validation = age_r1[validation]
age_test = age_r[test]

# bio
bio_data_r1 = bio_data_r[train1, :]
bio_data_train = bio_data_r1[train,:]
bio_data_validation = bio_data_r1[validation, :]
bio_data_test = bio_data_r[test,:]

cli_train=np.concatenate((np.matrix([age_train]).T, bio_data_train), axis=1)
cli_validation = np.concatenate((np.matrix([age_validation]).T, bio_data_validation), axis=1)
cli_test = np.concatenate((np.matrix([age_test]).T, bio_data_test), axis=1)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)  # rgb- add one more dimensionality
x_validation = x_validation.reshape(x_validation.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_validation = x_validation.astype('float32')
x_test = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_validation.shape[0], 'validation samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_validation = keras.utils.to_categorical(y_validation, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

image_input = Input(shape=input_shape)
other_data_input = Input(shape=(1,))

# First convolution
conv1 = Conv2D(32, kernel_size=(3, 3),
                activation='relu', padding='same'
                )(image_input)
# Second Convolution
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same'
                )(conv1)
conv2 = MaxPooling2D(pool_size=(4, 2))(conv2)
conv2 = Dropout(0.25)(conv2)
first_part_output = Flatten()(conv2)

#merged_model = keras.layers.concatenate([first_part_output, other_data_input])
# without age
merged_model = first_part_output
merged_model = Dense(64, activation='relu')(merged_model)
merged_model = Dropout(0.5)(merged_model)

predictions = Dense(num_classes, activation='softmax')(merged_model)

# Now create the model
model = Model(inputs=[image_input, other_data_input], outputs=predictions)
model.summary()
layers = model.layers

lr = 0.0001
beta_1 = 0.9
beta_2 = 0.999
optimizer = Adam(lr=lr, beta_1=beta_1, beta_2=beta_2)

model.compile(optimizer=optimizer, loss='binary_crossentropy',
              metrics=['accuracy'])

class_weighting = {0: 1, 1: 4.2}  # class weight
model.fit([x_train, age_train], [y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1, class_weight=class_weighting,
          validation_data=([x_validation, age_validation], [y_validation]))

score = model.predict([x_test, age_test], verbose=0)
model.save('PathCNN_model_noisy.h5')

observed = y_test
predicted = score

observed = y_test
predicted = score

auc = roc_auc_score(observed[:,0], predicted[:,0])
tn, fp, fn, tp = confusion_matrix(observed[:,0]>0.5, predicted[:,0]>0.5).ravel()
sen = tn/(tn+fp)
spe = tp/(tp+fn)
acc = (tp+tn)/(tp+tn+fp+fn)
print(f"AUC  Sen. Spe. Acc.\n{auc:0.2f} {sen:0.2f} {spe:0.2f} {acc:0.2f}")

Using TensorFlow backend.


x_train shape: (287, 146, 6, 1)
287 train samples
287 validation samples
287 test samples
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 146, 6, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 6, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 6, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 3, 64)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 3, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6912)              0         
___________________________________

In [ ]:
import os
import pandas as pd
import numpy as np
import keras.backend as K
import keras
import cv2

from keras.models import load_model

def grad_cam(model, category_index, layer_name):
    print(layer_name)

    inp = model.input
    y_c = model.output.op.inputs[0][0, category_index]
    A_k = model.get_layer(layer_name).output
    print(y_c, A_k)

    get_output = K.function(
        [inp], [A_k, K.gradients(y_c, A_k)[0], model.output])

    return get_output


def apply_grad_cam(get_output, input):
    input_image = input[0]

    [conv_output, grad_val, model_output] = get_output(input)

    conv_output = conv_output[0]
    grad_val = grad_val[0]

    weights = np.mean(grad_val, axis=(0, 1))

    grad_cam = np.zeros(dtype=np.float32, shape=conv_output.shape[0:2])
    for k, w in enumerate(weights):
        grad_cam += w * conv_output[:, :, k]

    grad_cam = np.maximum(grad_cam, 0)
    heatmap = grad_cam/np.max(grad_cam)

    image = input_image.squeeze()
    image -= np.min(image)
    image = 255 * image / np.max(image)
    image1 = cv2.resize(cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_GRAY2RGB), (240, 1460),
                        interpolation=cv2.INTER_NEAREST)

    print(np.percentile(heatmap/np.max(heatmap), [0, 25, 50, 75, 100]))
    heatmap1 = cv2.resize(heatmap/np.max(heatmap),
                          (240, 1460), interpolation=cv2.INTER_LINEAR)

    grad_cam = cv2.applyColorMap(np.uint8(255*heatmap1), cv2.COLORMAP_JET)
    grad_cam = np.float32(grad_cam) + np.float32(image1)
    grad_cam = 255 * grad_cam / np.max(grad_cam)

    return np.uint8(grad_cam), heatmap


pca_exp = pd.read_excel("data/PCA_EXP.xlsx", header=None)
pca_cnv = pd.read_excel("data/PCA_CNV.xlsx", header=None)
pca_mt = pd.read_excel("data/PCA_MT.xlsx", header=None)
clinical = pd.read_excel("data/Clinical.xlsx")

# data size: 146 pathways, 5 PCs
n = len(pca_exp)  # sample size: number of Pts
path_n = 146  # number of pathways
pc = 5  # number of PCs

# data creation-EXP
pca_exp = pca_exp.to_numpy()
print(pca_exp.shape)
exp_data = np.zeros((n, path_n, pc))
for i in range(n):
    for j in range(path_n):
        exp_data[i, j, :] = pca_exp[i, j * pc:(j + 1) * pc]

# data creation-CNV
pca_cnv = pca_cnv.to_numpy()
cnv_data = np.zeros((n, path_n, pc))
for i in range(n):
    for j in range(path_n):
        cnv_data[i, j, :] = pca_cnv[i, j * pc:(j + 1) * pc]

# data creation-MT
pca_mt = pca_mt.to_numpy()
mt_data = np.zeros((n, path_n, pc))
for i in range(n):
    for j in range(path_n):
        mt_data[i, j, :] = pca_mt[i, j * pc:(j + 1) * pc]

# data merge: mRNA expression, CNV, and MT with a specific number of PCs
no_pc = 2  # use the first 2 PCs among 5 PCs
all_data = np.zeros((n, path_n, no_pc * 3))
for i in range(n):
    all_data[i, :, :] = np.concatenate((exp_data[i, :, 0:no_pc], cnv_data[i, :, 0:no_pc], mt_data[i, :, 0:no_pc]),
                                       axis=1)

index = np.arange(0, pca_exp.shape[1], pc)

clinical = clinical.to_numpy()
survival = clinical[:, 5]
os_months = clinical[:, 6]
idx0 = np.where((survival == 1) & (os_months <= 24))  # class0
idx1 = np.where(os_months > 24)                       # class1

all_data = all_data[:, :, :]

data_0 = all_data[idx0, :, :]
data_0 = data_0[0, :, :, :]
data_1 = all_data[idx1, :, :]
data_1 = data_1[0, :, :, :]

outcomes_0 = np.zeros(len(idx0[0]))
outcomes_1 = np.ones(len(idx1[0]))

# data merge
data = np.concatenate((data_0, data_1))
outcomes = np.concatenate((outcomes_0, outcomes_1))

print('test')
event_rate = np.array(
    [outcomes_0.shape[0], outcomes_1.shape[0]])/outcomes_0.shape[0]
print(event_rate)

vmin = -10
vmax = 10
m = 1

model_file = 'PathCNN_model_noisy.h5'

model = load_model(model_file)
layers = model.layers
print(model.summary())

orig_weights = model.get_weights().copy()

output_layer = model.get_layer('dense_2')
output_weights = output_layer.get_weights()
print(output_weights[1])

col = ['outcome', 'prediction', 'target_class',
       'probability']+list(range(1, no_pc*path_n*3+1))
print(col)

layer = layers[4]  # the last layer before flatten
print(layer.name)
weights = layer.get_weights()

get_output = [None, None]
for target_class in range(2):
  get_output[target_class] = grad_cam(model, target_class, layer.name)

os.makedirs('output/'+layer.name, exist_ok=True)
for i in range(0, data.shape[0], 1):
    oimg = data[i, :, :]

    print(f'\ncase {i:03d}_{outcomes[i]:.0f}')
    print(oimg.shape)

    img = ((np.clip(oimg, vmin, vmax) - vmin) /
           (vmax - vmin) * 255).astype('uint8')

    timg = oimg.reshape(1, oimg.shape[0], oimg.shape[1], 1)

    preprocessed_input = [timg]

    model.set_weights(orig_weights)
    predictions = model.predict(preprocessed_input)
    print('Predicted class:')
    print(predictions)
    predicted_class = np.argmax(predictions)
    print(predicted_class)
    
    for target_class in range(2):
        gradcam, heatmap = apply_grad_cam(get_output[target_class], preprocessed_input)
        filename = f'output/{layer.name}/gradcam_noisy{i:03d}_{outcomes[i]:.0f}_{predicted_class:d}_{target_class:d}_{predictions[0][target_class]:0.2f}.png'
        #cv2.imwrite(filename, gradcam)

        heatmap1 = cv2.resize(heatmap, (no_pc*3, path_n))
        #print(heatmap.shape)
        data_row = [[outcomes[i], predicted_class, target_class,
                     predictions[0][target_class]]+heatmap1.flatten().tolist()]
        gradcam_pd = pd.DataFrame(data_row, columns=col, index=[i])
        if i == 0 and target_class == 0:
            gradcam_pd.to_csv(
                f'output/{layer.name}/pathcnn_gradcam_noisy.csv', mode='w')
        else:
            gradcam_pd.to_csv(f'output/{layer.name}/pathcnn_gradcam_noisy.csv',
                              mode='a', header=False)

(343, 730)
test
[1.         0.23706897]
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 146, 6, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 6, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 6, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 3, 64)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 3, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6912)              0         
_________________________________________________________________
dense_1 (Dense)    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


[nan nan nan nan nan]

case 001_0
(146, 6)
Predicted class:
[[0.33533257 0.6646674 ]]
1
[0.         0.02253579 0.11304542 0.25618409 1.        ]
[nan nan nan nan nan]

case 002_0
(146, 6)
Predicted class:
[[0.68605864 0.3139413 ]]
0
[0.         0.0511397  0.09848128 0.25029116 1.        ]
[nan nan nan nan nan]

case 003_0
(146, 6)
Predicted class:
[[0.5069816  0.49301842]]
0
[0.         0.03312664 0.07271915 0.18482433 1.        ]
[nan nan nan nan nan]

case 004_0
(146, 6)
Predicted class:
[[0.8669576 0.1330424]]
0
[0.         0.00351596 0.04346853 0.22270123 1.        ]
[nan nan nan nan nan]

case 005_0
(146, 6)
Predicted class:
[[0.6329292  0.36707073]]
0
[0.         0.01407394 0.05219999 0.24257816 1.        ]
[nan nan nan nan nan]

case 006_0
(146, 6)
Predicted class:
[[0.5103958 0.4896041]]
0
[0.         0.02071692 0.06827008 0.19405415 1.        ]
[nan nan nan nan nan]

case 007_0
(146, 6)
Predicted class:
[[0.3364542 0.6635458]]
1
[0.         0.02892565 0.09482861 0.26108321 1. 

## Comaparisons

